In [3]:
from bs4 import BeautifulSoup
import urllib
from profanity import profanity
from random import shuffle
import json

In [4]:
def make_url(search_string):
    search = search_string.replace(" ", "%20")
    full = "https://www.poetryfoundation.org/search?query=" + search + "&refinement=poems"
    return full

In [5]:
def get_links(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r, "lxml")
    links = soup.find_all("h2", class_="c-hdgSans c-hdgSans_2")
    for i in range(len(links)):
        links[i] = links[i].a["href"]
    return(links)

In [6]:
def get_poem(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r, "lxml")
    lines = []
    poem = soup.find("div", class_="o-poem")
    if poem:
        lines = poem.find_all("div")
    out = []
    if len(lines) > 300:
        lines = lines[:201]
    for line in lines:
        poem_line = line.get_text().replace(u'\r ', '').replace(u'\r', '')
        if all(ord(char) < 128 for char in poem_line) and not profanity.contains_profanity(poem_line):
            out.append(poem_line)
    return out

In [7]:
def get_poems_from_page(url):
    links = get_links(url)
    poems = []
    for link in links:
        poems.append(get_poem(link))
    return poems

In [8]:
def combine_poems(list1, list2):
    combined = list1 + list2
    shuffle(combined)
    combined = ["\n".join(x) for x in combined]
    combined = "\n".join(combined)
    return combined

In [9]:
def get_all_poems(search_string, num_pages):
    url = make_url(search_string)
    poems = []
    for i in range(1, num_pages + 1):
        page_poems = get_poems_from_page(url + '&page=' + str(i))
        poems += page_poems
    return poems

In [185]:
sheep_poems = get_all_poems('sheep', 11)

In [189]:
sheep_string = ["\n".join(x) for x in sheep_poems]
sheep_string = "\n".join(sheep_string)

In [191]:
with open("sheep-data/input2.txt", "w") as text_file:
    text_file.write(sheep_string)

In [10]:
def get_angst_poems(num_pages):
    url = 'https://www.poetryfoundation.org/ajax/poems?sort_by=recently_added&topics=6&school-period=1951-present'
    poems = []
    for i in range(1, num_pages + 1):
        page_link = url + '&page=' + str(i)
        r = urllib.urlopen(page_link).read()
        soup = BeautifulSoup(r, "lxml")
        full_json = soup.find("p")
        if full_json:
            full_json = full_json.get_text()
            full_json = json.loads(full_json)
            for j in range(0, len(full_json['Entries'])):
                link = full_json['Entries'][j]['link']
                poem = get_poem(link)
                poems.append(poem)
    return poems

In [11]:
angst_poems = get_angst_poems(60)

ValueError: Expecting ',' delimiter: line 1 column 18469 (char 18468)

In [ ]:
angst_string = ["\n".join(x) for x in angst_poems]
angst_string = "\n".join(angst_string)

In [ ]:
len(angst_string)

In [ ]:
with open("angst-data/input2.txt", "w") as text_file:
    text_file.write(angst_string)

u'https://www.poetryfoundation.org/poetrymagazine/poems/144809/the-orchestral-audition'

In [277]:
range(1, 10) + [11, 12, 13, 16, 17, 18, 

20